# Chapter 53: Contract Testing

---

## 53.1 What is Contract Testing?

Contract testing is a methodology for verifying that two independent services (usually a consumer and a provider) can communicate with each other as expected. Instead of running expensive end-to-end tests that exercise both services together, contract testing captures the interactions between them in a **contract** and verifies each side independently.

The contract defines the expectations: what requests the consumer will make and what responses the provider will return. The consumer side uses this contract to generate tests that verify it can handle the expected responses; the provider side uses the contract to verify it can produce those responses.

### 53.1.1 Why Contract Testing Matters

In a microservices architecture, services evolve independently. Without contract testing, integration issues can arise:

- A provider changes its API and breaks existing consumers.
- A consumer expects a field that the provider never sends.
- A provider adds a required field, but consumers don't send it.

End-to-end tests can catch some of these, but they are slow, brittle, and require all services to be deployed together. Contract testing provides fast, isolated feedback.

### 53.1.2 Consumer-Driven Contracts (CDC)

The most common form of contract testing is **consumer-driven contracts (CDC)**. In CDC, consumers define the contract based on their needs. This ensures that providers don't break existing consumers when they change.

The process:

1. **Consumer** writes an integration test that captures its expectations (e.g., "I will call GET /users/1 and expect a 200 response with a JSON body containing id and name").
2. The test framework generates a **contract file** (usually JSON or YAML) describing these interactions.
3. The contract is published to a **contract broker** (e.g., Pact Broker).
4. **Provider** retrieves the contract and runs verification tests against its actual implementation, ensuring it meets all consumer expectations.
5. If verification passes, both sides can deploy independently with confidence.

---

## 53.2 Consumer-Driven Contracts with Pact

Pact is the most widely adopted contract testing framework. It supports multiple languages and integrates with CI/CD.

### 53.2.1 Key Concepts

- **Consumer:** The service that makes HTTP requests.
- **Provider:** The service that responds to those requests.
- **Pact file:** A JSON document containing the interactions.
- **Pact Broker:** A repository for sharing pacts and verification results.

### 53.2.2 Pact Flow

```
┌────────┐     ┌──────────────┐     ┌────────┐
│Consumer│────▶│  Pact Broker │◀────│Provider│
└────────┘     └──────────────┘     └────────┘
     │                 │                  │
     │ Write pact      │ Store pact       │ Retrieve pact
     │                 │                   │
     ▼                 ▼                   ▼
Consumer tests    Pacts available     Provider verification
```

### 53.2.3 Consumer Side (JavaScript Example)

We'll implement a consumer (a web app) that calls a user service.

**Installation:**

```bash
npm install --save-dev @pact-foundation/pact
```

**Consumer test:**

```javascript
// consumer/userClient.spec.js
const { Pact } = require('@pact-foundation/pact');
const { getUser } = require('./userClient');
const path = require('path');

describe('UserService Client', () => {
  const provider = new Pact({
    consumer: 'WebApp',
    provider: 'UserService',
    port: 1234,
    log: path.resolve(process.cwd(), 'logs', 'pact.log'),
    dir: path.resolve(process.cwd(), 'pacts'),
    logLevel: 'INFO',
  });

  beforeAll(() => provider.setup());
  afterAll(() => provider.finalize());
  afterEach(() => provider.verify());

  describe('get user by ID', () => {
    test('should return a user with id 1', async () => {
      // Arrange: define expected interaction
      await provider.addInteraction({
        state: 'user exists',
        uponReceiving: 'a request for user 1',
        withRequest: {
          method: 'GET',
          path: '/users/1',
          headers: { Accept: 'application/json' },
        },
        willRespondWith: {
          status: 200,
          headers: { 'Content-Type': 'application/json' },
          body: {
            id: 1,
            name: 'John Doe',
            email: 'john@example.com',
          },
        },
      });

      // Act: call the real client (configured to use the mock provider)
      const user = await getUser(1, 'http://localhost:1234');

      // Assert
      expect(user).toEqual({
        id: 1,
        name: 'John Doe',
        email: 'john@example.com',
      });
    });
  });
});
```

**Consumer client code:**

```javascript
// consumer/userClient.js
const axios = require('axios');

async function getUser(id, baseURL) {
  const response = await axios.get(`${baseURL}/users/${id}`, {
    headers: { Accept: 'application/json' },
  });
  return response.data;
}

module.exports = { getUser };
```

When this test runs, it creates a pact file in `./pacts/webapp-userservice.json`.

### 53.2.4 Publishing the Pact

After tests pass, publish the pact to a Pact Broker.

```bash
npm install -g @pact-foundation/pact-cli
pact-broker publish ./pacts \
  --consumer-app-version 1.0.0 \
  --broker-base-url https://your-pact-broker.com \
  --broker-token your-token
```

### 53.2.5 Provider Side (Java Example with Spring Boot)

**Provider verification test:**

```java
// ProviderVerificationTest.java
import au.com.dius.pact.provider.junit5.PactVerificationContext;
import au.com.dius.pact.provider.junit5.PactVerificationInvocationContextProvider;
import au.com.dius.pact.provider.junitsupport.Provider;
import au.com.dius.pact.provider.junitsupport.State;
import au.com.dius.pact.provider.junitsupport.loader.PactBroker;
import au.com.dius.pact.provider.spring.junit5.MockMvcTestTarget;
import org.junit.jupiter.api.BeforeEach;
import org.junit.jupiter.api.TestTemplate;
import org.junit.jupiter.api.extension.ExtendWith;
import org.springframework.beans.factory.annotation.Autowired;
import org.springframework.boot.test.context.SpringBootTest;
import org.springframework.test.context.junit.jupiter.SpringExtension;

@Provider("UserService")
@PactBroker(url = "https://your-pact-broker.com", token = "your-token")
@SpringBootTest(webEnvironment = SpringBootTest.WebEnvironment.RANDOM_PORT)
@ExtendWith(SpringExtension.class)
public class ProviderVerificationTest {

    @Autowired
    private UserController userController;

    @TestTemplate
    @ExtendWith(PactVerificationInvocationContextProvider.class)
    void verifyPact(PactVerificationContext context) {
        context.verifyInteraction();
    }

    @BeforeEach
    void setUp(PactVerificationContext context) {
        MockMvcTestTarget testTarget = new MockMvcTestTarget();
        testTarget.setControllers(userController);
        context.setTarget(testTarget);
    }

    @State("user exists")
    public void userExists() {
        // Set up provider state: ensure user with ID 1 exists
        User user = new User(1, "John Doe", "john@example.com");
        userRepository.save(user);
    }
}
```

**Provider implementation:**

```java
// UserController.java
@RestController
public class UserController {
    @Autowired
    private UserRepository userRepository;

    @GetMapping("/users/{id}")
    public User getUser(@PathVariable Long id) {
        return userRepository.findById(id)
            .orElseThrow(() -> new UserNotFoundException());
    }
}
```

When the provider test runs, it fetches all pacts from the broker and verifies that the provider can satisfy them. If any expectation fails, the test fails.

### 53.2.6 Provider States

Provider states allow the provider to set up the necessary data for each interaction. In the example, the state `"user exists"` ensures that a user with ID 1 is present in the database.

---

## 53.3 Spring Cloud Contract

Spring Cloud Contract is an alternative contract testing framework for JVM languages, particularly well-suited for Spring Boot applications.

### 53.3.1 How It Works

- **Consumer** writes a contract (in Groovy or YAML) defining the request and expected response.
- The contract is placed in the consumer's `src/test/resources/contracts` directory.
- Spring Cloud Contract generates a test stub for the consumer and a verification test for the provider.
- The generated stub can be used in consumer tests to simulate the provider.

### 53.3.2 Contract Example (Groovy)

```groovy
// src/test/resources/contracts/user/getUser.groovy
import org.springframework.cloud.contract.spec.Contract

Contract.make {
    description "should return a user with id 1"
    request {
        method GET()
        url "/users/1"
        headers {
            accept applicationJson()
        }
    }
    response {
        status 200
        headers {
            contentType applicationJson()
        }
        body (
            id: 1,
            name: "John Doe",
            email: "john@example.com"
        )
    }
}
```

### 53.3.3 Provider Side

The provider includes the Spring Cloud Contract Verifier dependency and places contracts in its own `src/test/resources/contracts`. The plugin generates tests that verify the provider against the contracts.

```xml
<plugin>
    <groupId>org.springframework.cloud</groupId>
    <artifactId>spring-cloud-contract-maven-plugin</artifactId>
    <version>3.1.0</version>
    <extensions>true</extensions>
    <configuration>
        <baseClassForTests>
            com.example.UserContractBase
        </baseClassForTests>
    </configuration>
</plugin>
```

### 53.3.3 Consumer Side Stubs

The consumer can use the generated stub to test against a mock provider without running the actual provider. Stubs are published to a repository (e.g., Maven local) and used in consumer tests.

---

## 53.4 Contract Testing in Microservices

### 53.4.1 When to Use Contract Testing

- **Between services that communicate synchronously** (HTTP, gRPC).
- **When teams are independent** and deploy on different cadences.
- **When you want fast feedback** without full end-to-end tests.

### 53.4.2 Contract Testing vs. Integration Testing

| Aspect | Contract Testing | Integration Testing |
|--------|------------------|---------------------|
| **Scope** | Single interaction | Multiple services together |
| **Speed** | Fast (unit-test level) | Slow (deployment, setup) |
| **Reliability** | High (isolated) | Lower (flaky, environment issues) |
| **Feedback** | Early (in CI) | Late (after deployment) |
| **Maintenance** | Low (contracts versioned) | High (end-to-end suites) |

### 53.4.3 Contract Testing vs. Unit Testing

- **Unit tests** verify internal logic.
- **Contract tests** verify external boundaries.

They complement each other.

### 53.4.4 Contract Testing in CI/CD

- **Consumer CI:** Run consumer tests, publish pacts to broker.
- **Provider CI:** Retrieve latest pacts, run verification. If verification passes, provider can deploy.
- **Can-i-deploy:** Use `pact-broker can-i-deploy` to check if a service version is compatible with its consumers/providers.

```bash
pact-broker can-i-deploy \
  --pacticipant UserService \
  --version 1.2.3 \
  --to-environment production \
  --broker-base-url https://broker
```

This checks all contracts involving UserService version 1.2.3 and ensures they are verified.

---

## 53.5 Best Practices

### 53.5.1 Keep Contracts Focused

Each contract should represent a single interaction. Don't bundle multiple unrelated requests.

### 53.5.2 Use Provider States

Provider states allow the provider to set up the right conditions. This keeps tests deterministic.

### 53.5.3 Version Contracts

Always version your contracts along with the service code. The pact file includes the consumer and provider names and versions.

### 53.5.4 Automate Verification

Run provider verification in CI on every change. Fail the build if verification fails.

### 53.5.5 Use a Pact Broker

A central broker helps manage contracts across teams, provides versioning, and enables `can-i-deploy` checks.

### 53.5.6 Don't Overuse

Not every interaction needs a contract test. Start with critical paths and expand as needed.

---

## 53.6 Common Challenges and Solutions

| Challenge | Solution |
|-----------|----------|
| **Provider state setup is complex** | Use test data factories; mock external dependencies. |
| **Contract changes frequently** | Establish team communication; use tags to manage versions. |
| **Pact files become large** | Split into multiple pacts per consumer. |
| **Broker management overhead** | Use hosted services (PactFlow) if self-hosting is difficult. |
| **Asynchronous communication** | Pact supports message queues (Pact for messaging). |

---

## Chapter Summary

In this chapter, we explored **contract testing**:

- **What it is** – verifying service interactions using consumer-driven contracts.
- **Why it's essential** in microservices to prevent integration issues.
- **Pact framework** – the industry standard for contract testing with examples in JavaScript (consumer) and Java (provider).
- **Spring Cloud Contract** – a JVM-specific alternative.
- **Integration with CI/CD** using Pact Broker and `can-i-deploy`.
- **Best practices** and **common challenges**.

**Key Insight:** Contract testing provides fast, reliable feedback on service compatibility, enabling independent deployments and reducing reliance on brittle end-to-end tests.

---

## 📖 Next Chapter: Chapter 54 - Chaos Engineering

Now that you can ensure your services communicate correctly, Chapter 54 introduces **Chaos Engineering**—the practice of intentionally injecting failures to test system resilience.